In [1]:
from IPython.display import Image

## demo

$$
x-y\leq Mz\\
x-y\geq -Mz
$$

- $z\in (0, 1)$
- $z=0$ => $x=y$ 
- $z=1$ => $-M\leq x-y\leq M$
    - indicating that the variables x and y can have any values so long as the absolute value of their difference is bounded by $M$ (hence the need for M to be "large enough.")

In [3]:
Image(url='https://files.realpython.com/media/lp-py-eq-4.0178c4cfe357.png', width=400)

In [6]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus

In [7]:
# Define the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables (dict)
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

# Set the objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/whaow/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/afb24c8e45d64858a80c9a9eccd8727b-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/afb24c8e45d64858a80c9a9eccd8727b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 23 RHS
At line 27 BOUNDS
At line 28 ENDATA
Problem MODEL has 3 rows, 4 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 4 (0) columns and 10 (0) elements
0  Obj -0 Dual inf 97 (4)
0  Obj -0 Dual inf 97 (4)
2  Obj 1900
Optimal - objective value 1900
Optimal objective 1900 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

status: 1, Optimal
objective: 1900.0
x1: 5.0
x2: 0.0
x3: 45.0
x4: 0.0
manpower: 0

- 现在如果要求 x1 和 x3 不能并行生产（不能既生产 x1 也成产 x3)
$$
y[1]+y[3]\leq 1
$$

In [8]:
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
y = {i: LpVariable(name=f"y{i}", cat="Binary") for i in (1, 3)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

M = 100

model += (y[1] + y[3] <= 1, "y_constraint")
model += (x[1] <= y[1] * M, "x1_constraint")
model += (x[3] <= y[3] * M, "x3_constraint")


# Set objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/whaow/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/da8b3b87d46f429dbed843e04ee92319-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/da8b3b87d46f429dbed843e04ee92319-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 36 RHS
At line 43 BOUNDS
At line 46 ENDATA
Problem MODEL has 6 rows, 6 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1900 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 6 columns (2 integer (2 of which binary)) and 16 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 4.44089e-16
Cbc0038I Solution found of -1800
Cbc0038I Relaxing continuous gives -1800
Cbc0038I Before mini branch and bound, 2 inte